In [1]:
import sys
import collections
import re

from laf.fabric import LafFabric
from etcbc.preprocess import prepare
fabric = LafFabric(verbose = '')

  0.00s This is LAF-Fabric 4.5.21
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [2]:
API = fabric.load('etcbc4b','--','word_order_verbless',

                  {'primary': False,
                   'xmlids':{'node':False,'edge':False},
                   'features':('book chapter verse otype code typ txt monads function g_cons_utf8',''),
                   'prepare': prepare
        
                    }
)
exec(fabric.localnames.format(var='fabric'))

  0.00s LOADING API: please wait ... 
  0.06s USING main  DATA COMPILED AT: 2015-11-02T15-08-56
  4.30s LOGFILE=/Users/Cody/laf-fabric-output/etcbc4b/word_order_verbless/__log__word_order_verbless.txt
  4.30s INFO: LOADING PREPARED data: please wait ... 
  4.30s prep prep: G.node_sort
  4.43s prep prep: G.node_sort_inv
  4.96s prep prep: L.node_up
  8.88s prep prep: L.node_down
    15s prep prep: V.verses
    15s prep prep: V.books_la
    15s ETCBC reference: http://laf-fabric.readthedocs.org/en/latest/texts/ETCBC-reference.html
    17s INFO: LOADED PREPARED data
    17s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX lexicon FOR TASK word_order_verbless AT 2016-03-30T01-42-07


In [3]:
def get_percent(total,freq):
    return str(round(float(freq / total)*100,2))+'%'

def get_ref(ref_type,node):
    if ref_type == 'verse':
        verse = L.u('verse',node)
        return F.verse.v(verse)
    elif ref_type == 'chapter':
        chapter = L.u('chapter',node)
        return F.chapter.v(chapter)
    elif ref_type == 'book':
        book = L.u('book',node)
        return F.book.v(book)

# In Song

In [4]:
#indexes all nodes in corpus for faster searches

corpus = ["Canticum"]

cur_book = None
nodes = []

for n in NN():
    if cur_book in corpus:
        nodes.append(n)
    
    if F.otype.v(n) == 'book':
        cur_book = F.book.v(n)
        
msg('{} nodes appended'.format(len(nodes)))

    27s 6020 nodes appended


In [5]:
verbless = ['NmCl','AjCl']

verbless_phrases = []

for n in nodes:
    otype = F.otype.v(n)
    if otype == 'clause' and F.typ.v(n) in verbless:
        verbless_phrases.append(L.d('phrase',n))

In [6]:
verbless_subjects = []

for phrase in verbless_phrases:
    new_phrase= []
    subject = False
    preC = False
    for word in phrase:
        if F.function.v(word) == 'Subj':
            subject = True
    for word in phrase:
        if F.function.v(word) == 'PreC':
            preC = True
    if subject == True and preC == True:
        for word in phrase:
            new_phrase.append(F.function.v(word))
        verbless_subjects.append(new_phrase)

In [7]:
subj_first = []
predC_first = []

subj_first_total = 0
predC_first_total = 0
total = 0

for phrase in verbless_subjects:
    total += 1
    for function in phrase:
        if function == 'Subj':
            subj_first_total += 1
            subj_first.append(phrase)
            break
        elif function == 'PreC':
            predC_first_total += 1
            predC_first.append(phrase)
            break

In [8]:
print ('total verbless with subjects:   {}'.format(total))
print ('total verbless with subj first: {} ({})'.format(subj_first_total,get_percent(total,subj_first_total)))
print ('total verbless with preC first: {} ({})'.format(predC_first_total,get_percent(total,predC_first_total)))


total verbless with subjects:   90
total verbless with subj first: 65 (72.22%)
total verbless with preC first: 25 (27.78%)


# Verbless Mains (Song)

In [10]:
def code_filter(node):
    
    code = int(F.code.v(node))
    
    #asyndetic
    if 100 <= code <= 167:
        return True
    
    #parallel
    elif 200 <= code <= 201:
        return True
    
    #asyndetic with conj.
    elif 300 <= code <= 367:
        return True
        
    #syndetic
    elif 400 <= code <= 487:
        return True
    
    #first cl in direct speech
    elif code == 999:
        return True

In [13]:
verbless = ['NmCl','AjCl']

verbless_phrases = []

for n in nodes:
    otype = F.otype.v(n)
    if otype == 'clause_atom' and F.typ.v(n) in verbless and code_filter(n) == True:
        verbless_phrases.append(L.d('phrase',n))

In [14]:
verbless_subjects = []

for phrase in verbless_phrases:
    new_phrase= []
    subject = False
    preC = False
    for word in phrase:
        if F.function.v(word) == 'Subj':
            subject = True
    for word in phrase:
        if F.function.v(word) == 'PreC':
            preC = True
    if subject == True and preC == True:
        for word in phrase:
            new_phrase.append(F.function.v(word))
        verbless_subjects.append(new_phrase)

In [15]:
subj_first = []
predC_first = []

subj_first_total = 0
predC_first_total = 0
total = 0

for phrase in verbless_subjects:
    total += 1
    for function in phrase:
        if function == 'Subj':
            subj_first_total += 1
            subj_first.append(phrase)
            break
        elif function == 'PreC':
            predC_first_total += 1
            predC_first.append(phrase)
            break

In [16]:
print ('total verbless with subjects:   {}'.format(total))
print ('total verbless with subj first: {} ({})'.format(subj_first_total,get_percent(total,subj_first_total)))
print ('total verbless with preC first: {} ({})'.format(predC_first_total,get_percent(total,predC_first_total)))


total verbless with subjects:   78
total verbless with subj first: 59 (75.64%)
total verbless with preC first: 19 (24.36%)


# In Ancient Poetry

In [68]:
corpus = {'Genesis_49':[str(x) for x in range (1,28)],
          'Exodus_15':[str(x) for x in range (1,22)],
          'Numeri_23':[str(x) for x in range (7,11)]+[str(x) for x in range (18,25)],
          'Numeri_24':[str(x) for x in range (3,10)]+[str(x) for x in range (15,25)],
          'Deuteronomium_32':[str(x) for x in range (1,44)],
          'Deuteronomium_33':[str(x) for x in range (2,30)],
          'Judices_5':[str(x) for x in range (2,32)]
         }

cur_book = None
poetry_nodes = []

for n in NN():
    chapter = get_ref('chapter',n)
    verse = get_ref('verse',n)
    if ('{}_{}'.format(cur_book,chapter) in corpus) and verse in corpus['{}_{}'.format(cur_book,chapter)]:
        poetry_nodes.append(n)
    
    if F.otype.v(n) == 'book':
        cur_book = F.book.v(n)
        
msg('{} nodes appended'.format(len(poetry_nodes)))

22m 09s 9523 nodes appended


In [70]:
verbless = ['NmCl','AjCl']

verbless_phrases = []

for n in poetry_nodes:
    otype = F.otype.v(n)
    if otype == 'clause' and F.typ.v(n) in verbless:
        verbless_phrases.append(L.d('phrase',n))

In [71]:
verbless_subjects = []

for phrase in verbless_phrases:
    new_phrase= []
    subject = False
    preC = False
    for word in phrase:
        if F.function.v(word) == 'Subj':
            subject = True
    for word in phrase:
        if F.function.v(word) == 'PreC':
            preC = True
    if subject == True and preC == True:
        for word in phrase:
            new_phrase.append(F.function.v(word))
        verbless_subjects.append(new_phrase)

In [72]:
subj_first = []
predC_first = []

subj_first_total = 0
predC_first_total = 0
total = 0

for phrase in verbless_subjects:
    total += 1
    for function in phrase:
        if function == 'Subj':
            subj_first_total += 1
            subj_first.append(phrase)
            break
        elif function == 'PreC':
            predC_first_total += 1
            predC_first.append(phrase)
            break

In [74]:
print ('total verbless with subjects:   {}'.format(total))
print ('total verbless with subj first: {} ({})'.format(subj_first_total,get_percent(total,subj_first_total)))
print ('total verbless with preC first: {} ({})'.format(predC_first_total,get_percent(total,predC_first_total)))

total verbless with subjects:   85
total verbless with subj first: 49 (57.65%)
total verbless with preC first: 36 (42.35%)


# In Psalms

In [75]:
#indexes all nodes in corpus for faster searches

corpus = ["Psalmi"]

cur_book = None
psalms_nodes = []

for n in NN():
    if cur_book in corpus:
        psalms_nodes.append(n)
    
    if F.otype.v(n) == 'book':
        cur_book = F.book.v(n)
        
msg('{} nodes appended'.format(len(psalms_nodes)))

27m 57s 99028 nodes appended


In [76]:
verbless = ['NmCl','AjCl']

verbless_phrases = []

for n in psalms_nodes:
    otype = F.otype.v(n)
    if otype == 'clause' and F.typ.v(n) in verbless:
        verbless_phrases.append(L.d('phrase',n))

In [77]:
verbless_subjects = []

for phrase in verbless_phrases:
    new_phrase= []
    subject = False
    preC = False
    for word in phrase:
        if F.function.v(word) == 'Subj':
            subject = True
    for word in phrase:
        if F.function.v(word) == 'PreC':
            preC = True
    if subject == True and preC == True:
        for word in phrase:
            new_phrase.append(F.function.v(word))
        verbless_subjects.append(new_phrase)

In [78]:
subj_first = []
predC_first = []

subj_first_total = 0
predC_first_total = 0
total = 0

for phrase in verbless_subjects:
    total += 1
    for function in phrase:
        if function == 'Subj':
            subj_first_total += 1
            subj_first.append(phrase)
            break
        elif function == 'PreC':
            predC_first_total += 1
            predC_first.append(phrase)
            break

In [79]:
print ('total verbless with subjects:   {}'.format(total))
print ('total verbless with subj first: {} ({})'.format(subj_first_total,get_percent(total,subj_first_total)))
print ('total verbless with preC first: {} ({})'.format(predC_first_total,get_percent(total,predC_first_total)))

total verbless with subjects:   645
total verbless with subj first: 342 (53.02%)
total verbless with preC first: 303 (46.98%)
